### Documentação em: 
https://developers.google.com/sheets/api/quickstart/python 

In [1]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

### Planilha Google:
https://docs.google.com/spreadsheets/d/18iQHIDJ1_CVMK1vRUOmxtAaMCKUjEre5CqcARTAZGRU/edit#gid=851668008

In [2]:
# Recuperando arquivo e configuracoes da planilha:
import json
with open('config.json', 'r') as f:
    data = json.load(f)

# print(data)

In [3]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
# SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = data['SAMPLE_SPREADSHEET_ID']


# 1-ACAO; 2-FII
TIPO = 1 

# Acoes:
if TIPO == 1:
    SAMPLE_RANGE_NAME = 'PRECOTETO!A1:D26'
    UPDATE_RANGE_NAME = 'PRECOTETO!D1'
# FIIs:
if TIPO == 2:
    SAMPLE_RANGE_NAME = 'PRECOTETO!A27:D47'
    UPDATE_RANGE_NAME = 'PRECOTETO!D27'

# Parametros:
PARMS_RANGE_NAME = 'PRECOTETO!G1:H2'


#### lOGIN:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

#### APLICACAO:
try:
    # Padrao:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()

    # Recebendo informacoes da planilha:
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()        

    # Recebendo parametros para outros calculos:
    # Recebendo informacoes da planilha:
    parms = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=PARMS_RANGE_NAME).execute()        

    # Result é um dicionario:
    # print(result)

    valores = result['values']
    print(valores)

    parms = parms['values']
    print(parms)
               
    
except HttpError as err:
    print(err)


[['Papel', 'SufixoPapel', 'Cotacao', 'PrecoTeto'], ['BBAS3', '.SA', 'R$ 34,66'], ['VIVT3', '.SA', 'R$ 37,83'], ['TAEE11', '.SA', 'R$ 34,73'], ['KLBN11', '.SA', 'R$ 19,84'], ['SAPR11', '.SA', 'R$ 18,17'], ['BBSE3', '.SA', 'R$ 33,51'], ['CPLE11', '.SA', 'R$ 38,35']]
[['PeriodoDividendosEmAnos', 'PercentualRetorno'], ['5,0', '6,0']]


In [4]:
# Input das Informações Necessárias:
for i, linha in enumerate(parms):    
    if i > 0:
        anos = float(linha[0].replace("R$ ", "").replace(".", "").replace(",", "."))
        percentual = float(linha[1].replace("R$ ", "").replace(".", "").replace(",", ".")) / 100
# print(anos, percentual)
       

In [5]:
# Data Início = Data Fim (Hoje) - anos:
from datetime import date, timedelta
dtfim = date.today()
dtini = dtfim - (anos * timedelta(365))

# print(dtfim, dtini)

### Yahoo Finance

In [6]:
# pip install yfinance 

In [7]:
# Iniciar lista de Preco Teto que serah adicionada aa planilha:
VALORES_ADICIONAR = [
    [valores[0][3]],
]

print(VALORES_ADICIONAR)

[['PrecoTeto']]


In [8]:
# Buscar dados do papel e adicionar Preco Teto:
import yfinance as yf
import pandas as pd

for i, linha in enumerate(valores):    
    if i > 0:
        papel = linha[0] + linha[1]
        precoTeto = 0.0
        
        # -----        
        try:
            df = yf.download(papel, start=dtini, end=dtfim, actions=True)
            
            if hasattr(df, 'Dividends'):            
                dividendos = df.loc[df.Dividends != 0, 'Dividends']
                dividendos = dividendos.reset_index(level=0)
                print(dividendos)
                
                dividendosAnual = dividendos.groupby(pd.Grouper(key='Date', freq='Y')).sum()
                dividendosAnual = dividendosAnual.reset_index(level=0)
                print(dividendosAnual)

                media = dividendosAnual['Dividends'].mean()
                
                precoTeto = media / percentual                                
            
        except HttpError as err:
            print(err)
        
        # -----
        
        VALORES_ADICIONAR.append([precoTeto])
        
print(VALORES_ADICIONAR)

C:\Users\Work\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


[*********************100%***********************]  1 of 1 completed
         Date  Dividends
0  2018-03-02   0.245670
1  2018-03-13   0.081710
2  2018-05-22   0.213954
3  2018-06-12   0.077203
4  2018-08-22   0.269662
5  2018-09-12   0.085491
6  2018-11-22   0.416893
7  2018-12-12   0.125670
8  2019-02-22   0.591888
9  2019-03-07   0.590146
10 2019-03-12   0.156124
11 2019-05-22   0.414838
12 2019-06-12   0.171054
13 2019-08-22   0.446155
14 2019-09-12   0.232955
15 2019-11-22   0.365042
16 2019-12-12   0.176174
17 2020-02-26   0.438882
18 2020-03-12   0.181420
19 2020-08-24   0.441988
20 2020-09-14   0.102862
21 2020-11-17   0.194773
22 2020-12-14   0.116984
23 2021-02-23   0.435863
24 2021-03-12   0.145714
25 2021-05-24   0.414445
26 2021-06-14   0.168519
27 2021-08-24   0.348288
28 2021-09-14   0.184739
29 2021-11-23   0.393703
30 2021-12-14   0.174991
31 2022-03-03   0.824721
32 2022-03-15   0.210628
33 2022-05-24   0.155347
34 2022-06-14   0.250285
35 2022-08-23   0.204622
36 202

In [9]:
# Editando informacoes na planilha:
result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=UPDATE_RANGE_NAME,valueInputOption="USER_ENTERED",
                              body={'values': VALORES_ADICIONAR}).execute()